### Import library

In [30]:
import ace_lib as ace
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px
import pygwalker as pyg
import glob
import re

### Start session
Enter credentials once - they will be saved to local folder and loaded each time

In [9]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_wJtwPQQGiBkcysVKY2LoYfeGDH8L

 


## Scoring

In [ ]:
def score(raw_data):
    score_data = raw_data[['test.turnover', 'test.returns', 'test.drawdown']]
    score = pd.DataFrame()
    score = 1.3673 + 0.1757*score_data['test.turnover'] + 0.6150*score_data['test.returns'] - 1.0816*score_data['test.drawdown']
    return score

## Alpha Mixing Templates

In [18]:
'''
signal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));

signal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));

signal1 * (1 + signal2)
'''

'\nsignal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));\n\nsignal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));\n\nsignal1 * (1 + signal2)\n'

In [19]:
'''
signal1 = oth176_lmt_close_eq_score ;
signal2 = oth176_malta_eq_score ;
k = ts_corr(signal1, signal2, 30) ;
k*regression_neut(signal1, signal2)

[아이디어]
경향성이 비슷한 알파 활용하여 corr로 weight를 부여
'''

'\nsignal1 = oth176_lmt_close_eq_score ;\nsignal2 = oth176_malta_eq_score ;\nk = ts_corr(signal1, signal2, 30) ;\nk*regression_neut(signal1, signal2)\n\n[아이디어]\n경향성이 비슷한 알파 활용하여 corr로 weight를 부여\n'

## Read all passed alphas

In [77]:
file_direc = glob.glob('files/signal/corr/*')
file_direc

['files/signal/corr\\pt_daedalus.csv',
 'files/signal/corr\\pt_template_A_clustering_result_corr.csv',
 'files/signal/corr\\pt_template_C_clustering_result_corr.csv',
 'files/signal/corr\\pt_template_D_clustering_result_corr.csv']

In [84]:
pd.read_csv(file_direc[1])['converted_expression']

0     trade_when(oth460_1l_acda<=0.5, regression_neu...
1     trade_when(oth460_1l_ccns<=0.5, regression_neu...
2     trade_when(oth460_1l_ccns<=0.5, vector_neut(ts...
3     trade_when(oth460_1l_acda<=0.5, vector_neut(ts...
4     trade_when(oth460_1l_m3r_mtn_spb_se<=0.5, regr...
                            ...                        
75    trade_when(oth460_1l_ccns<=0.5, regression_neu...
76    trade_when(oth460_1l_acda<=0.5, vector_neut(ts...
77    trade_when(oth460_1l_acda<=0.5, regression_neu...
78    trade_when(oth460_1l_acda<=0.5, vector_neut(ts...
79    trade_when(oth460_1l_acda<=0.5, regression_neu...
Name: converted_expression, Length: 80, dtype: object

In [87]:
expressions = {}
expressions['daedalus'] = pd.read_csv(file_direc[0])['expression']
expressions['A'] = pd.read_csv(file_direc[1])['converted_expression']
expressions['C'] = pd.read_csv(file_direc[2])['converted_expression']
expressions['D'] = pd.read_csv(file_direc[3])['converted_expression']

## Template A mixing

In [94]:
a = expressions['A']
a[0]

'trade_when(oth460_1l_acda<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_acda>0.5)'

In [ ]:
'''
signal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));

signal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));

signal1 * (1 + signal2)
'''

### add

In [96]:
alpha_list = []

for i in range(10):
    alpha_list.append(ace.generate_alpha(f'add({a[i]}, {a[i+1]}, filter = false)', region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01))

alpha_list

[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 0,
   'neutralization': 'MARKET',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'add(trade_when(oth460_1l_acda<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_acda>0.5), trade_when(oth460_1l_ccns<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cpss_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_ccns>0.5), filter = false)'},
 {'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 0,
   'neutralization': 'MARKET',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'u

In [ ]:
result = ace.simulate_alpha_list_multi(s, alpha_list)

In [ ]:
result

In [ ]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

In [ ]:
result_st1.to_csv('A_mixing_add.csv')

### mult

In [99]:
'''
signal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));

signal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));

signal1 * (1 + signal2)
'''

'\nsignal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));\n\nsignal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));\n\nsignal1 * (1 + signal2)\n'

In [102]:
alpha_list = []

for i in range(10):
    alpha_list.append(ace.generate_alpha(f'{a[2*i]}*(1+ {a[2*i+1]})', region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01))

alpha_list

[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 0,
   'neutralization': 'MARKET',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'trade_when(oth460_1l_acda<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_acda>0.5)*(1+ trade_when(oth460_1l_ccns<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cpss_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_ccns>0.5))'},
 {'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 0,
   'neutralization': 'MARKET',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VE

In [ ]:
result = ace.simulate_alpha_list_multi(s, alpha_list)

 50%|█████████████████████████████████████████▌                                         | 2/4 [18:00<15:47, 473.78s/it]

### corr

In [58]:
alpha_list = []

for i in range(10):
    alpha_list.append(ace.generate_alpha(f'k = ts_corr({a[i]}, {a[i+1]}, 30);k*regression_neut({a[i]}, {a[i+1]})', region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01))

alpha_list



[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 0,
   'neutralization': 'MARKET',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'k = ts_corr(trade_when(oth460_1l_acda<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_acda>0.5), trade_when(oth460_1l_ccns<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cpss_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_ccns>0.5), 30);k*regression_neut(trade_when(oth460_1l_acda<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_acda>0.5), trade_when(oth460_1l_ccns<=0.5, regression_neut(ts_zscore(ts_backfill(

In [63]:
result = ace.simulate_alpha_list_multi(s, alpha_list)

 25%|████████████████████▊                                                              | 1/4 [03:07<09:22, 187.53s/it]

An error occurred


 50%|█████████████████████████████████████████▌                                         | 2/4 [04:58<04:45, 142.54s/it]

An error occurred


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [05:34<01:33, 93.81s/it]

An error occurred


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [19:01<00:00, 285.34s/it]

An error occurred
Your simulation has been running too long. If you are running simulations in batch, consider to reduce number of concurrent simulations.


In [65]:
result

[{'alpha_id': None,
  'simulate_data': {'type': 'REGULAR',
   'settings': {'instrumentType': 'EQUITY',
    'region': 'GLB',
    'universe': 'MINVOL1M',
    'delay': 1,
    'decay': 0,
    'neutralization': 'MARKET',
    'truncation': 0.01,
    'pasteurization': 'ON',
    'testPeriod': 'P0Y0M0D',
    'unitHandling': 'VERIFY',
    'nanHandling': 'OFF',
    'language': 'FASTEXPR',
    'visualization': False},
   'regular': 'k = ts_corr(trade_when(oth460_1l_acda<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_acda>0.5), trade_when(oth460_1l_ccns<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cpss_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_ccns>0.5), 30);k*regression_neut(trade_when(oth460_1l_acda<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_acda>0.5), trade_when(oth460_

In [64]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

ValueError: No objects to concatenate

In [62]:
result_st1.to_csv('A_mixing_add.csv')

In [51]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(result)
is_tests_df.head()

,alpha_id,competitions,limit,name,result,value
0,roV9mgo,NaN,1.25,LOW_SHARPE,PASS,2.6200
1,roV9mgo,NaN,1.00,LOW_FITNESS,PASS,1.5400
2,roV9mgo,NaN,0.01,LOW_TURNOVER,PASS,0.4315
3,roV9mgo,NaN,0.70,HIGH_TURNOVER,PASS,0.4315
4,roV9mgo,NaN,NaN,CONCENTRATED_WEIGHT,PASS,NaN


In [52]:
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

Failed alphas:[]
Passed alphas:['Vp5Zog5', 'EwLNb7m', 'm81deY5', 'roV9mgo', 'z90gz11', 'MpZdXOr', 'ngQmo9M', 'k3OveNk', 'QagOmKg', 'lV9ERan']


In [60]:
#calling submit_alpha on all alphas that have passed the submission tests
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['summer_boot_camp_4조'])

In [59]:
ace.get_check_submission(s, passed_alphas[3])

,name,result,limit,value,competitions,alpha_id
0,LOW_SHARPE,PASS,1.25,2.6200,NaN,roV9mgo
1,LOW_FITNESS,PASS,1.00,1.5400,NaN,roV9mgo
2,LOW_TURNOVER,PASS,0.01,0.4315,NaN,roV9mgo
3,HIGH_TURNOVER,PASS,0.70,0.4315,NaN,roV9mgo
4,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,roV9mgo
5,LOW_SUB_UNIVERSE_SHARPE,PASS,0.92,1.2600,NaN,roV9mgo
6,SELF_CORRELATION,FAIL,0.70,0.8117,NaN,roV9mgo
7,PROD_CORRELATION,PENDING,0.70,NaN,NaN,roV9mgo
8,REGULAR_SUBMISSION,PASS,4.00,0.0000,NaN,roV9mgo
9,MATCHES_COMPETITION,PASS,NaN,NaN,"[{'id': 'challenge', 'name': 'Challenge'}]",roV9mgo


In [ ]:
submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas}

## Template C Mixing

In [83]:
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['summer_boot_camp_4조'])

In [87]:
ace.get_check_submission(s, passed_alphas[0])

,name,result,limit,value,competitions,alpha_id
0,LOW_SHARPE,PASS,1.25,2.0200,NaN,oKP2Wrl
1,LOW_FITNESS,PASS,1.00,1.1200,NaN,oKP2Wrl
2,LOW_TURNOVER,PASS,0.01,0.3259,NaN,oKP2Wrl
3,HIGH_TURNOVER,PASS,0.70,0.3259,NaN,oKP2Wrl
4,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,oKP2Wrl
5,LOW_SUB_UNIVERSE_SHARPE,PASS,0.87,0.9200,NaN,oKP2Wrl
6,SELF_CORRELATION,PASS,0.70,0.4739,NaN,oKP2Wrl
7,PROD_CORRELATION,PASS,0.70,0.6252,NaN,oKP2Wrl
8,REGULAR_SUBMISSION,PASS,4.00,3.0000,NaN,oKP2Wrl
9,MATCHES_COMPETITION,PASS,NaN,NaN,"[{'id': 'challenge', 'name': 'Challenge'}]",oKP2Wrl


When you got a list of submittable alphas, you can call function submit_alpha()

In [89]:
#calling submit_alpha on all alphas that have passed the submission tests
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['summer_boot_camp_4조'])
submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas}

In [90]:
#submit_result will have return values from the submit_result function

submit_result

{'oKP2Wrl': True}

### Library Fuctions.

following are some other functions that you can use for your own analysis

**get_alpha_pnl(s, alpha_id)** - to get the pnl for an alpha

**get_alpha_yearly_stats(s, alpha_id)** - to get yearly statistics for an alpha

**get_self_corr(s, alpha_id)** - to get self correlation results for an alpha

**get_prod_corr(s, alpha_id)** - to get prod correlation results for an alpha

**get_check_submission(s, alpha_id)** - to get check submission result for an alpha

**check_self_corr_test(s, alpha_id)** - to check if alpha passes self correlation test (self_corr<0.7)

**check_prod_corr_test(s, alpha_id)** - to check if alpha passes prod correlation test (prod_corr<0.7)

**perfomance_comparison(s, alpha_id)** - to get the result of performance comparison for an alpha merged performance

In [37]:
ace.get_alpha_pnl(s, result[0]['alpha_id'])

,Pnl,alpha_id
Date,,
2012-01-23,0.0,dx9ALqJ
2012-01-24,52190.0,dx9ALqJ
2012-01-25,32038.0,dx9ALqJ
2012-01-26,37403.0,dx9ALqJ
2012-01-27,53631.0,dx9ALqJ
...,...,...
2022-01-17,12935472.0,dx9ALqJ
2022-01-18,12935661.0,dx9ALqJ
2022-01-19,12967474.0,dx9ALqJ
